In [ ]:
## import libraries
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore') #for ignoring warnings

pd.set_option('display.max_columns', None) #display all columns for better view and understanding in-place

In [ ]:
#reading the data
loan_df = pd.read_csv("loan.csv")

In [ ]:
loan_df.head()

In [ ]:
loan_df.shape

## Data Cleaning 


In [ ]:
loan_df.isnull().sum() #get no of null values in each column

### Lots of columns with all null values , removing them ....

In [ ]:
loan_df.dropna(axis = 1, how = 'all', inplace = True) 

In [ ]:
loan_df.head()

In [ ]:
loan_df.columns[loan_df.nunique() <= 1] #Get single valued columns

### There are some single valued columns as well , we need to remove them also ...

In [ ]:
loan_df.drop(['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med',
       'policy_code', 'application_type', 'acc_now_delinq',
       'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens'], axis=1, inplace=True)

In [ ]:
loan_df.shape

### Drop the columns which have no significance to the analysis 

- Some columns are generated post the approval of loan and since we are analyzing the factors contributing to defaulting before approving the loan , we can simply remove these columns.

- Example of post features are delinq_2yrs, revol_bal, out_prncp, total_pymnt, total_rec_prncp, total_rec_int, total_rec_late_fee, recoveries, collection_recovery_fee, last_pymnt_d, last_pymnt_amnt, next_pymnt_d, chargeoff_within_12_mths, mths_since_last_delinq, mths_since_last_record

- Other columns like member_id, id, title , emp_title, desc, zip_code, addr_state, last_credit_pull_d don't contribute to analysis since they are irrelevant.

- funded_amnt is not needed since we only need the actual amount and same is provide by funded_amnt_inv.

In [ ]:
loan_df.drop(['delinq_2yrs', 'revol_bal', 'out_prncp', 'total_pymnt', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'mths_since_last_delinq', 'mths_since_last_record', 'desc', 'member_id', 'id', 'title', 'emp_title', 'zip_code', 'addr_state', 'url', 'next_pymnt_d', 'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
'funded_amnt'], axis=1, inplace=True)

In [ ]:
loan_df.shape

### We are left with 24 columns, let's come to rows analysis now ...

Existing/Current loans data won't contribute to the analysis of whether the new payee will default or not , we need to delete these rows also ..

In [ ]:
loan_df.drop(loan_df[loan_df['loan_status'] == 'Current'].index, axis=0, inplace=True)

In [ ]:
loan_df.shape
loan_df.loan_status.unique()

### Identify the Categorical columns 

In [ ]:
data = []
for col in loan_df.columns.tolist():
    data.append({'column': col, 'count': len(loan_df[col].unique())})

In [ ]:
unique = pd.DataFrame(data).sort_values('count')
unique.head()

### Deriving new columns

In [ ]:
loan_df['term_months'] = loan_df['term'].str.rstrip('months')

In [ ]:
loan_df['int_rate_percentage'] = loan_df['int_rate'].str.rstrip('%')

In [ ]:
loan_df['emp_length_years'] = loan_df['emp_length'].str.rstrip('years ')
loan_df['emp_length_years'].replace({'10+': '10'}, inplace=True)
loan_df['emp_length_years'][loan_df['emp_length_years'] == '< 1'] = '0.5'

In [ ]:
loan_df['revol_util_percentage'] = loan_df['revol_util'].str.rstrip('%')

In [ ]:
loan_df['issue_d_month'], loan_df['issue_d_year'] = loan_df['issue_d'].str.split('-').str
loan_df['earliest_cr_line_month'], loan_df['earliest_cr_line_year'] = loan_df['earliest_cr_line'].str.split('-').str

In [ ]:
loan_df['issue_d_year'] = '20' + loan_df['issue_d_year']

In [ ]:
loan_df.issue_d_year

In [ ]:
columns = ['loan_amnt','funded_amnt_inv', 'installment', 'annual_inc', 'emp_length_years', 'dti','out_prncp_inv', 'total_pymnt_inv', 'int_rate_percentage', 'emp_length_years','revol_util_percentage']
for col in columns :
    loan_df[col] = loan_df[col].astype(np.float)

In [ ]:
# Creating a profit and loss column
loan_df['Profil_Loss']=(
    loan_df['total_pymnt_inv']-loan_df['funded_amnt_inv']
)*100/loan_df['funded_amnt_inv']

## Outlier Treatment

In [ ]:
sns.boxplot(loan_df['annual_inc'])

- There are clearly outliers in annual income , we need to treat them
- Check for various quantiles present

In [ ]:
quantile_info = loan_df.quantile([0.5, 0.75,0.90, 0.95, 0.97,0.98, 0.99])
quantile_info['annual_inc']

- The values after 95 percentile are disconnected from other data distribution , only 0-95th percentile data may make sense in this case

In [ ]:
per_95_annual_inc = loan_df['annual_inc'].quantile(0.95)
loan_df = loan_df[loan_df.annual_inc <= per_95_annual_inc]

In [ ]:
sns.boxplot(loan_df['annual_inc'])

- Annual income column is fixed now

#### Let's analyse other columns 

- dti
- loan_amnt
- funded_amnt_inv

In [ ]:
sns.boxplot(loan_df.dti)

In [ ]:
sns.boxplot(loan_df.loan_amnt)

In [ ]:
sns.boxplot(loan_df.funded_amnt_inv)

- Though there are some values far from distribution in case of loan_amnt & funded_amnt_inv, the distribution is still contigous and there is no need to remove outliers.

## Univariate Analysis / Categorical Analysis of Data

In [ ]:
sns.boxplot( y=loan_df["loan_amnt"] )
plt.ylabel('Loan Amount')
plt.show()

<b>Inference</b> : The loan amount varies from 0 to 35,000 having mean of 10,000

#### Since we need to find the parameters that can derive the loan defaulters , let's consider the univariate analysis for loans which have 'Charged Off' loan status

In [ ]:
sns.countplot(x = 'loan_status', data = loan_df)

In [ ]:
sns.countplot(x = 'grade', data = loan_df[loan_df.loan_status == 'Charged Off'], order = ['A', 'B', 'C', 'D', 'E', 'F', 'G'])

In [ ]:
loan_df.sub_grade_numeric = pd.to_numeric(loan_df.sub_grade.apply(lambda x : x[-1]))
fig, ax = plt.subplots(figsize=(15,8))
sns.countplot(x = 'grade', order = ['A', 'B', 'C', 'D', 'E', 'F', 'G'] , hue = 'sub_grade',data = loan_df[loan_df.loan_status == 'Charged Off'])

#### Home ownership analysis

In [ ]:
# unique values for home_ownership
loan_df['home_ownership'].unique()

In [ ]:
plt.subplots(figsize = (8,5))
#ax.set(yscale = 'log')
sns.countplot(x='home_ownership', data=loan_df[loan_df['loan_status']=='Charged Off'])

<b> Inference: </b> : There are more chances of defaulting when the applicant is living in rented house

#### Purpose analysis

In [ ]:
fig, ax = plt.subplots(figsize = (14,8))
sns.countplot(y ='purpose', data=loan_df[loan_df.loan_status == 'Charged Off'])

<b> Inference: </b> : Debt consolidation purpose has highest chances of defaults 

In [ ]:
sns.countplot(y='term', data=loan_df[loan_df['loan_status']=='Charged Off'])

<b>Inference</b> : It seems that loans having 36 months term have more defaults

In [ ]:
sns.countplot(x='verification_status', data=loan_df[loan_df['loan_status']=='Charged Off'])

<b>Inference</b> : Loans which has source verified have lesser number of defaults

In [ ]:
fig,ax = plt.subplots(figsize = (10,8))
ax.set_yscale('log')
sns.countplot(x='inq_last_6mths', data=loan_df[loan_df['loan_status']=='Charged Off'])

#### Analyzing by issued month and year

In [ ]:
loan_df.head()

In [ ]:
plt.figure(figsize=(16,16))
plt.subplot(221)
sns.countplot(x='issue_d_month', data=loan_df[loan_df['loan_status']=='Charged Off'])
plt.subplot(222)
sns.countplot(x='issue_d_year', data=loan_df[loan_df['loan_status']=='Charged Off'])

<b>Inference: </b>Maximum number of defaults have occured when the loan was sanctioned in Dec. Defaults for loans issued in the year 2011 were also high as compared to other years


#### Analyzing installment,dti, loan_amnt

In [ ]:
loan_df['installment_groups'] = pd.cut(loan_df['installment'], bins=10,precision =0,labels=['14-145','145-274','274-403','403-531','531-660','660-789','789-918','918-1047','1047-1176','1176-1305'])
loan_df['funded_amnt_inv_group'] = pd.cut(loan_df['funded_amnt_inv'], bins=7,labels=['0-5k','5k-10k','10k-15k','15k-20k','20k-25k','25k-30k','30k-35k'])
loan_df['loan_amnt_groups'] = pd.cut(loan_df['loan_amnt'], bins=7,precision =0,labels=['0-5k','5k-10k','10k-15k','15k-20k','20k-25k','25k-30k','30k-35k'])
loan_df['dti_groups'] = pd.cut(loan_df['dti'], bins=5,precision =0,labels=['0-6','6-12','12-18','18-24','24-30'])

In [ ]:
fig,ax = plt.subplots(figsize = (14,5))
ax.set_yscale('log')
sns.countplot(x='funded_amnt_inv_group', data=loan_df[loan_df['loan_status']=='Charged Off'])

In [ ]:
fig,ax = plt.subplots(figsize = (15,6))
ax.set_yscale('log')
sns.countplot(x='loan_amnt_groups', data=loan_df[loan_df['loan_status']=='Charged Off'])

In [ ]:
sns.countplot(x='dti_groups', data=loan_df[loan_df['loan_status']=='Charged Off'])

In [ ]:
fig,ax = plt.subplots(figsize = (15,6))
ax.set_yscale('log')
sns.countplot(x='installment_groups', data=loan_df[loan_df['loan_status']=='Charged Off'])

In [ ]:
sns.countplot(x='emp_length_years', data=loan_df[loan_df['loan_status'] == 'Charged Off'])
plt.xlabel('Employee Working Experience')
plt.show()

<b> Inference </b> : Employees with 10 or more years of experience have more chances of defaulting...

In [ ]:
loan_df['int_rate_groups'] = pd.cut(loan_df['int_rate_percentage'], bins=5,precision =0,labels=['5%-9%','9%-13%','13%-17%','17%-21%','21%-24%'])

In [ ]:
fig, ax = plt.subplots(figsize = (15,10))
plt.subplot(221)
sns.countplot(x='int_rate_groups', data=loan_df[loan_df.loan_status == 'Charged Off'])
plt.xlabel('Interest Rate')

<b> Inference: </b> Loans with interest rate 13-17% have highest number of defaults.

In [ ]:
loan_df['annual_inc_groups'] = pd.cut(loan_df['annual_inc'], bins=5,precision =0,labels =['3k-31k','31k-58k','58k-85k','85k-112k','112k-140k'])

In [ ]:
fig, ax = plt.subplots(figsize = (10,6))
sns.countplot(x='annual_inc_groups', data=loan_df[loan_df.loan_status == 'Charged Off'])

<b>Inference: </b> Applicants who have an income of range 31201 - 58402 have higher chance of defaulting

### Cumulative Observations

#### With above plots, we noticed that there are more chances of defaulting when :

- Applicants having house_ownership as 'RENT'
- Applicants who receive interest at the rate of 13-17%
- Applicants who have an income of range 31201 - 58402
- Applicants with employement length of 10 or more
- When funded amount by investor is between 5000-10000
- Loan amount is between 5429 - 10357
- Dti is between 12-18
- When monthly installments are between 145-274
- Term of 36 months
- When the loan status is Not verified
- When the no of enquiries in last 6 months is 0
- When the purpose is 'debt_consolidation'
- Grade is 'B'
- And a total grade of 'B5' level.

### Annual Income vs other columns analysis 

#### Plot of annual income vs purpose

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(data =loan_df,x='annual_inc', y='purpose', hue ='loan_status',palette="deep")
plt.show()

debt_consolidation had highest number of default as seen earlier but the annual income is not highest in that case , instead it
is highest in case of home_improvement purpose

Applicants with higher salary mostly applied loans for "home_improvment", "house", "renewable_energy" and "small_businesses"

#### Plot for Annual income vs home ownership

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(data =loan_df,x='annual_inc', y='home_ownership', hue ='loan_status')
plt.show()

Nothing substantial could be drawn in this case however in all the categories fully paid loan are more than defaults/charged_off loans

#### Annual Income vs Loan amount

In [ ]:
sns.barplot(x = "annual_inc_groups", y = "loan_amnt", hue = 'loan_status', data = loan_df)

<b> Inference: </b> The loan_amount is higher for people who defaulted among all income groups ...


#### Annual income vs int_rate

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(data =loan_df,x='int_rate_groups', y='annual_inc', hue ='loan_status')
plt.show()

### Loan Amount vs other column analysis 

#### Plot for Loan Amount vs Loan Purpose

In [ ]:
plt.figure(figsize=(12,12))
sns.barplot(data =loan_df,x='loan_amnt', y='purpose', hue ='loan_status')
plt.show()

#### Plot for loan vs house ownership

In [ ]:
plt.figure(figsize=(12,12))
sns.barplot(data =loan_df,x='loan_amnt', y='home_ownership', hue ='loan_status')
plt.show()

#### Plot for Loan amount vs Grade

In [ ]:
plt.figure(figsize=(12,12))
sns.barplot(data =loan_df,x='loan_amnt', y='grade', hue ='loan_status', order=['A','B','C','D','E','F','G'])
plt.show()